In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
df1 = pd.read_csv("globalterrorismdb_0718dist.csv", encoding='ISO-8859-1')


C:\Users\acer\AppData\Local\Temp\ipykernel_10736\3615890850.py:4: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("globalterrorismdb_0718dist.csv", encoding='ISO-8859-1')


In [2]:
merged_df1 = df1[df1['iyear'].isin([2014,2015,2016,2017])]

In [3]:
df= merged_df1.groupby(['iday', 'imonth','iyear', 'country_txt', 'region_txt', 
    'provstate', 'targtype1_txt', 'targsubtype1_txt',
    'corp1',  'weaptype1_txt', 'weapsubtype1_txt','gname' , 'claimmode_txt']).size().reset_index()

In [45]:
df2= merged_df1.groupby(['iday', 'imonth','iyear', 'country_txt', 'region_txt', 
    'provstate', 'targtype1_txt', 'targsubtype1_txt',
    'corp1',  'weaptype1_txt', 'weapsubtype1_txt','gname' , 'claimmode_txt']).size().reset_index()

In [4]:
df.columns = df.columns.astype(str)

In [5]:
value_counts = df['gname'].value_counts()
classes_to_keep = value_counts[value_counts > 0].index

df = df[df['gname'].isin(classes_to_keep)]

threshold = 50
rare_classes = value_counts[value_counts < threshold].index

df['gname'] = df['gname'].replace(rare_classes, 'Other')

df['gname'].value_counts()


gname
Other                                             2243
Islamic State of Iraq and the Levant (ISIL)       1749
Taliban                                           1460
Al-Shabaab                                         891
Tehrik-i-Taliban Pakistan (TTP)                    333
New People's Army (NPA)                            253
Sinai Province of the Islamic State                244
Baloch Republican Army (BRA)                       187
Khorasan Chapter of the Islamic State              177
Baloch Liberation Front (BLF)                      138
Al-Qaida in the Arabian Peninsula (AQAP)           131
Communist Party of India - Maoist (CPI-Maoist)     117
Maoists                                            113
Unknown                                            108
Houthi extremists (Ansar Allah)                    104
Al-Nusrah Front                                     80
Boko Haram                                          78
Baloch Liberation Army (BLA)                        73
Trip

In [46]:
value_counts = df2['gname'].value_counts()
classes_to_keep = value_counts[value_counts > 0].index

df2 = df2[df2['gname'].isin(classes_to_keep)]

threshold = 50
rare_classes = value_counts[value_counts < threshold].index

df2['gname'] = df2['gname'].replace(rare_classes, 'Other')

df2['gname'].value_counts()

gname
Other                                             2243
Islamic State of Iraq and the Levant (ISIL)       1749
Taliban                                           1460
Al-Shabaab                                         891
Tehrik-i-Taliban Pakistan (TTP)                    333
New People's Army (NPA)                            253
Sinai Province of the Islamic State                244
Baloch Republican Army (BRA)                       187
Khorasan Chapter of the Islamic State              177
Baloch Liberation Front (BLF)                      138
Al-Qaida in the Arabian Peninsula (AQAP)           131
Communist Party of India - Maoist (CPI-Maoist)     117
Maoists                                            113
Unknown                                            108
Houthi extremists (Ansar Allah)                    104
Al-Nusrah Front                                     80
Boko Haram                                          78
Baloch Liberation Army (BLA)                        73
Trip

In [6]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [7]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embeddings(texts):
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

def process_in_batches(texts, batch_size=32):
    embeddings_list = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_embeddings = get_embeddings(batch_texts)
        embeddings_list.append(batch_embeddings)
    return np.vstack(embeddings_list)

def apply_pca(embeddings, n_components=50):
    pca = PCA(n_components=n_components)
    reduced_embeddings = pca.fit_transform(embeddings)
    return reduced_embeddings,pca

for col in categorical_columns:
    print(f"Processing column: {col}")
    text_data = df[col].astype(str).tolist()  
    embeddings = process_in_batches(text_data, batch_size=32)
    
    reduced_embeddings,pca = apply_pca(embeddings, n_components=50)  

    embeddings_df = pd.DataFrame(reduced_embeddings, columns=[f'{col}_embedding_{i}' for i in range(reduced_embeddings.shape[1])])
    df = pd.concat([df, embeddings_df], axis=1)


df.drop(columns=categorical_columns, inplace=True)

df.head()


Processing column: country_txt
Processing column: region_txt
Processing column: provstate
Processing column: targtype1_txt
Processing column: targsubtype1_txt
Processing column: corp1
Processing column: weaptype1_txt
Processing column: weapsubtype1_txt
Processing column: gname
Processing column: claimmode_txt


,iday,imonth,iyear,0,country_txt_embedding_0,country_txt_embedding_1,country_txt_embedding_2,country_txt_embedding_3,country_txt_embedding_4,country_txt_embedding_5,...,claimmode_txt_embedding_40,claimmode_txt_embedding_41,claimmode_txt_embedding_42,claimmode_txt_embedding_43,claimmode_txt_embedding_44,claimmode_txt_embedding_45,claimmode_txt_embedding_46,claimmode_txt_embedding_47,claimmode_txt_embedding_48,claimmode_txt_embedding_49
0,1,1,2014,1,0.630217,-2.276473,-1.391857,1.380639,-0.721671,-1.029484,...,9.591287e-07,0.000002,1.049153e-07,-4.059460e-07,7.787272e-08,9.558599e-07,-8.829945e-07,3.457248e-07,-2.050922e-07,2.183359e-07
1,1,1,2014,1,0.630208,-2.276454,-1.391824,1.380682,-0.721680,-1.029451,...,-4.731028e-07,-0.000001,-1.245703e-06,-3.624268e-07,5.011415e-06,5.779097e-06,-7.258962e-06,2.640680e-06,-1.691649e-06,-3.441137e-06
2,1,1,2014,2,0.502387,0.086141,0.736830,-0.726387,2.238811,-0.601764,...,-2.256920e-06,0.000004,-1.146405e-06,-2.511584e-06,2.732902e-06,2.796937e-06,-7.874209e-06,-7.378201e-08,-3.250088e-06,-3.585386e-07
3,1,1,2014,1,0.502387,0.086141,0.736850,-0.726369,2.238787,-0.601756,...,4.120844e-07,-0.000004,2.979288e-06,-2.544321e-06,-2.485128e-06,8.549834e-06,2.581929e-06,5.978625e-07,2.004250e-06,-2.923756e-06
4,1,1,2015,1,-0.806723,2.077236,-1.493703,1.007914,0.531483,0.396596,...,9.509242e-08,0.000001,2.807726e-06,6.272170e-06,1.567720e-06,-7.541312e-06,7.278508e-06,-3.703475e-06,-1.242381e-06,2.554112e-06


In [8]:
train = df[df['iyear'].isin([2014,2015,2016])]
test = df[df['iyear'].isin([2017])]

In [9]:
gname_pca_columns = [f'gname_embedding_{i}' for i in range(50)]

y_train = train[gname_pca_columns]
x_train = train.drop(columns=gname_pca_columns)

y_test = test[gname_pca_columns]
x_test = test.drop(columns=gname_pca_columns)

In [10]:
x_train.columns = x_train.columns.astype(str)
x_test.columns = x_test.columns.astype(str)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler
import numpy as np
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=42) 
y_train_clustered = kmeans.fit_predict(y_train)
y_test_clustered = kmeans.predict(y_test)


C:\Program Files\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [13]:
r = RandomForestClassifier(n_estimators=200 ,bootstrap =False , max_depth= None ,random_state=42, class_weight= 'balanced')
r.fit(x_train,y_train_clustered)

RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       n_estimators=200, random_state=42)

In [14]:
y_pred = r.predict(x_test)

In [15]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print("Accuracy:", accuracy_score(y_test_clustered, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test_clustered, y_pred))
print("Classification Report:\n", classification_report(y_test_clustered, y_pred,zero_division=0))

Accuracy: 0.8730544747081712
Confusion Matrix:
 [[440  21   0   1   0   2   0   0   6   0]
 [ 15 422   7  11   8   4   1  10   6   3]
 [  0   1 272   0   0   0   0   1   0   5]
 [  2   0   0 314   0   0   0   0   0   0]
 [  0  26  47   0  95   4   0  10   0   0]
 [  2  10   0   0   0  43   0   3   0   0]
 [  0   6   0   0   0   0  41   0   0   0]
 [  0  16   1   0   1   2   0  58   0   0]
 [  0   5   0   0   0   4   0   0  96   0]
 [  0   1  19   0   0   0   0   0   0  14]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.94      0.95       470
           1       0.83      0.87      0.85       487
           2       0.79      0.97      0.87       279
           3       0.96      0.99      0.98       316
           4       0.91      0.52      0.66       182
           5       0.73      0.74      0.74        58
           6       0.98      0.87      0.92        47
           7       0.71      0.74      0.72        78
          